In [1]:
from lightrag.components.agent import ReActAgent
from lightrag.core import Generator, ModelClientType, ModelClient
from lightrag.utils import setup_env, get_logger

# get_logger(level="DEBUG")

setup_env()


# Define tools
def multiply(a: int, b: int) -> int:
    """
    Multiply two numbers.
    """
    return a * b


async def add(a: int, b: int) -> int:
    """
    Add two numbers.
    """
    return a + b


def divide(a: float, b: float) -> float:
    """
    Divide two numbers.
    """
    return float(a) / b


llama3_model_kwargs = {
    "model": "llama3-8b-8192",  # llama3 70b works better than 8b here.
    "temperature": 0.0,
}
gpt_model_kwargs = {
    "model": "gpt-3.5-turbo",
    "temperature": 0.0,
}


def test_react_agent(model_client: ModelClient, model_kwargs: dict):
    tools = [multiply, add, divide]
    queries = [
        # "What is the capital of France? and what is 465 times 321 then add 95297 and then divide by 13.2?",
        "Give me 5 words rhyming with cool, and make a 4-sentence poem using them",
    ]
    # define a generator without tools for comparison

    generator = Generator(
        model_client=model_client,
        model_kwargs=model_kwargs,
    )

    react = ReActAgent(
        max_steps=6,
        add_llm_as_fallback=True,
        tools=tools,
        model_client=model_client,
        model_kwargs=model_kwargs,
    )
    # print(react)

    for query in queries:
        print(f"Query: {query}")
        agent_response = react.call(query)
        llm_response = generator.call(prompt_kwargs={"input_str": query})
        print(f"Agent response: {agent_response}")
        print(f"LLM response: {llm_response}")
        print("")


def test_react_agent_use_examples(model_client: ModelClient, model_kwargs: dict):
    tools = [multiply, add, divide]
    queries = [
        "What is the capital of France? and what is 465 times 321 then add 95297 and then divide by 13.2?",
        "Give me 5 words rhyming with cool, and make a 4-sentence poem using them",
    ]

    from lightrag.core.types import FunctionExpression

    # add examples for the output format str
    example_using_multiply = FunctionExpression.from_function(
        func=multiply,
        thought="Now, let's multiply two numbers.",
        a=3,
        b=4,
    )
    react = ReActAgent(
        max_steps=6,
        add_llm_as_fallback=True,
        tools=tools,
        model_client=model_client,
        model_kwargs=model_kwargs,
        examples=[example_using_multiply],
    )

    print(react)

    # see the output format
    react.planner.print_prompt()

    for query in queries:
        print(f"Query: {query}")
        agent_response = react.call(query)
        print(f"Agent response: {agent_response}")
        print("")


if __name__ == "__main__":
    test_react_agent(ModelClientType.GROQ(), llama3_model_kwargs)
    test_react_agent(ModelClientType.OPENAI(), gpt_model_kwargs)
    print("Done")

    test_react_agent_use_examples(ModelClientType.GROQ(), llama3_model_kwargs)


Query: Give me 5 words rhyming with cool, and make a 4-sentence poem using them
2024-07-16 13:39:37 - [react.py:280:call] - input_query: Give me 5 words rhyming with cool, and make a 4-sentence poem using them
2024-07-16 13:39:38 - [react.py:260:_run_one_step] - Step 1: 
StepOutput(step=1, action=FunctionExpression(thought="I need to generate 5 words rhyming with 'cool' and create a poem.", action='llm_tool(input="What are 5 words that rhyme with \'cool\'?"'), function=Function(thought=None, name='llm_tool', args=[], kwargs={'input': "What are 5 words that rhyme with 'cool'?"}), observation='Here are 5 words that rhyme with "cool":\n\n1. Rule\n2. Tool\n3. Fool\n4. School\n5. Pool')
_______

2024-07-16 13:39:38 - [react.py:260:_run_one_step] - Step 2: 
StepOutput(step=2, action=FunctionExpression(thought="I need to create a 4-sentence poem using the 5 words that rhyme with 'cool'.", action='llm_tool(input="Can you help me create a 4-sentence poem using the words \'rule\', \'tool\', \'fo

Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})
Error executing FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})
/Users/liyin/Documents/test/LightRAG/lightrag/lightrag/components/agent/react.py:234: RuntimeWarning: coroutine 'FunctionTool.acall' was never awaited
  return action_step


2024-07-16 13:39:46 - [react.py:260:_run_one_step] - Step 3: 
StepOutput(step=3, action=FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'), function=None, observation='Error executing FunctionExpression(thought="Now, let\'s add 95297 to the result.", action=\'add(a=149265, b=95297)\'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name=\'add\', args=[], kwargs={\'a\': 149265, \'b\': 95297})')
_______



Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})
Error executing FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})


2024-07-16 13:39:47 - [react.py:260:_run_one_step] - Step 4: 
StepOutput(step=4, action=FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'), function=None, observation='Error executing FunctionExpression(thought="Now, let\'s add 95297 to the result.", action=\'add(a=149265, b=95297)\'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name=\'add\', args=[], kwargs={\'a\': 149265, \'b\': 95297})')
_______



...

2024-07-16 13:39:47 - [react.py:260:_run_one_step] - Step 5: 
StepOutput(step=5, action=FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'), function=None, observation='Error executing FunctionExpression(thought="Now, let\'s add 95297 to the result.", action=\'add(a=149265, b=95297)\'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name=\'add\', args=[], kwargs={\'a\': 149265, \'b\': 95297})')
_______



Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})
Error executing FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})


2024-07-16 13:39:47 - [react.py:260:_run_one_step] - Step 6: 
StepOutput(step=6, action=FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'), function=None, observation='Error executing FunctionExpression(thought="Now, let\'s add 95297 to the result.", action=\'add(a=149265, b=95297)\'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name=\'add\', args=[], kwargs={\'a\': 149265, \'b\': 95297})')
_______

2024-07-16 13:39:47 - [react.py:294:call] - answer:
 Error executing FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'): Error asyncio.run() cannot be called from a running event loop executing function: Function(thought=None, name='add', args=[], kwargs={'a': 149265, 'b': 95297})
Agent response: Error executing FunctionExpression(thought="Now, let's add 95297 to the result.", action='add(a=149265, b=95297)'): Error asyncio.run() canno

Error at parsing JSON string: Got invalid JSON object with yaml.safe_load. Error: while parsing a flow mapping
  in "<unicode string>", line 1, column 1:
    {
    ^
expected ',' or '}', but got '<scalar>'
  in "<unicode string>", line 3, column 107:
     ... ule, Tool, Fool, School, Pool')")
                                         ^. Got JSON string: {
    "thought": "Now, let's create a poem using the rhyming words.",
    "action": "llm_tool(input='Create a 4-sentence poem using the words: Rule, Tool, Fool, School, Pool')")
}
Error in parsing JSON to JSON: Error: Got invalid JSON object with yaml.safe_load. Error: while parsing a flow mapping
  in "<unicode string>", line 1, column 1:
    {
    ^
expected ',' or '}', but got '<scalar>'
  in "<unicode string>", line 3, column 107:
     ... ule, Tool, Fool, School, Pool')")
                                         ^. Got JSON string: {
    "thought": "Now, let's create a poem using the rhyming words.",
    "action": "llm_tool(input='C

2024-07-16 13:39:48 - [react.py:294:call] - answer:
 Error planning step 2: Error: Got invalid JSON object with yaml.safe_load. Error: while parsing a flow mapping
  in "<unicode string>", line 1, column 1:
    {
    ^
expected ',' or '}', but got '<scalar>'
  in "<unicode string>", line 3, column 107:
     ... ule, Tool, Fool, School, Pool')")
                                         ^. Got JSON string: {
    "thought": "Now, let's create a poem using the rhyming words.",
    "action": "llm_tool(input='Create a 4-sentence poem using the words: Rule, Tool, Fool, School, Pool')")
}
Agent response: Error planning step 2: Error: Got invalid JSON object with yaml.safe_load. Error: while parsing a flow mapping
  in "<unicode string>", line 1, column 1:
    {
    ^
expected ',' or '}', but got '<scalar>'
  in "<unicode string>", line 3, column 107:
     ... ule, Tool, Fool, School, Pool')")
                                         ^. Got JSON string: {
    "thought": "Now, let's create a poe

In [6]:
# test async

import asyncio
import time

def is_running_in_event_loop() -> bool:
    try:
        loop = asyncio.get_running_loop()
        if loop.is_running():
            return True
        else:
            return False
    except RuntimeError:
        return False
    
def sync_func():
    time.sleep(1)
    print("Sync function")


def execute():
    if is_running_in_event_loop():
        print("Running in event loop")
        try:
            result = asyncio.to_thread(sync_func)
            print("Result", result)
        except RuntimeError:
            print("Runtime error")
            sync_func()
    else:
        sync_func()

In [7]:
execute()

Running in event loop
Result <coroutine object to_thread at 0x1351c1e40>


/var/folders/h8/nhgbdr4d18x2r49j4pk5z6gw0000gn/T/ipykernel_98631/3325042598.py:1: RuntimeWarning: coroutine 'to_thread' was never awaited
  execute()
